In [ ]:
import sys
import time
import os
import glob
from time import sleep
import datetime
import pytz
import argparse

import numpy as np
import pandas as pd
import pickle
import joblib

import scipy # 여기에 오타가 나있었네 어째 불안하다
import sklearn
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

from scipy.stats import skew, kurtosis
#from Current_Feature_Extractor import Extract_Time_Features, Extract_Phase_Features, Extract_Freq_Features

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Extract_Time_Features:    
    def __init__(self, rawTimeData):
        self._TimeFeatures = []
        self._rawTimeData = rawTimeData

    def AbsMax(self):
        self._absmax = np.abs(self._rawTimeData).max(axis=1)
        return self._absmax
    
    def AbsMean(self):
        self._absmean = np.abs(self._rawTimeData).mean(axis=1)
        return self._absmean
    
    def P2P(self):
        self._peak2peak = np.max(self._rawTimeData,axis=1) - np.min(self._rawTimeData,axis=1)
        return self._peak2peak
    
    def Skewness(self):
        self._skewness = skew(self._rawTimeData, axis=1, nan_policy='raise')
        return self._skewness
    
    def Kurtosis(self):
        self._kurtosis = kurtosis(self._rawTimeData, axis=1, fisher=False)
        return self._kurtosis

    def RMS(self):
        self._rms = np.sqrt(np.sum(self._rawTimeData**2, axis=1) / self._rawTimeData.shape[1])
        return self._rms
    
    def CrestFactor(self):
        self._cresfactor = self.P2P() / self.RMS()
        return self._cresfactor
    
    def ShapeFactor(self):
        self._shapefactor = self.RMS() / self.AbsMean()
        return self._shapefactor
    
    def ImpulseFactor(self):
        self._impulsefactor = self.AbsMax() / self.AbsMean()
        return self._impulsefactor

    def Features(self):
        # Time-domain Features #
        self._TimeFeatures.append(self.AbsMax())        # Feature 1: Absolute Maximum 
        self._TimeFeatures.append(self.AbsMean())       # Feature 2: Absolute Mean
        self._TimeFeatures.append(self.P2P())           # Feature 3: Peak-to-Peak
        self._TimeFeatures.append(self.RMS())           # Feature 4: Root-mean-square
        self._TimeFeatures.append(self.Skewness())      # Feature 5: Skewness
        self._TimeFeatures.append(self.Kurtosis())      # Feature 6: Kurtosis
        self._TimeFeatures.append(self.CrestFactor())   # Feature 7: Crest Factor
        self._TimeFeatures.append(self.ShapeFactor())   # Feature 8: Shape Factor
        self._TimeFeatures.append(self.ImpulseFactor()) # Feature 9: Impulse Factor
                
        return np.asarray(self._TimeFeatures)

In [ ]:
#%%  Define class for phase feature extraction ##
class Extract_Phase_Features:    
    def __init__(self, rawTimeData, Fs):
        self._PhaseFeatures = []
        self._rawTimeData = rawTimeData - np.expand_dims(np.mean(rawTimeData,axis=1),axis=1) # Raw time data
        self._Fs = Fs                    # Sampling frequency [Hz]
        
        # Phase Shift #
    
    def Shift(self):
        
        _r = self._rawTimeData[0]
        _s = self._rawTimeData[1]
        _t = self._rawTimeData[2]
        
        _N = self._rawTimeData.shape[1]
        
        _t = np.linspace(0.0, ((_N - 1) / self._Fs), _N)
        _dts = np.linspace(-_t[-1], _t[-1], (2 * _N) - 1)
        
        # RS Phase shift
        self._RS_corr = np.correlate(_r, _s, 'full')
        self._RS_t = _dts[self._RS_corr.argmax()]
        self._RS_phase = ((2.0 * np.pi) * ((self._RS_t / (1.0 / self._Fs)) % 1.0)) - np.pi
        
        # ST Phase shift
        self._ST_corr = np.correlate(_s, _t, 'full')
        self._ST_t = _dts[self._ST_corr.argmax()]
        self._ST_phase = ((2.0 * np.pi) * ((self._ST_t / (1.0 / self._Fs)) % 1.0)) - np.pi
        
         # TR Phase shift
        self._TR_corr = np.correlate(_t, _r, 'full')
        self._TR_t = _dts[self._TR_corr.argmax()]
        self._TR_phase = ((2.0 * np.pi) * ((self._TR_t / (1.0 / self._Fs)) % 1.0)) - np.pi
        
        # Phase Level shift
        self._RS_level = _r.max() - _s.max()
        self._ST_level = _s.max() - _t.max()
        self._TR_level = _t.max() - _r.max()
        
        return (self._RS_phase, self._ST_phase, self._TR_phase, self._RS_level, self._ST_level, self._TR_level)
    
    
    def Features(self):
        shift = self.Shift()
        
        self._PhaseFeatures.append(shift[0])
        self._PhaseFeatures.append(shift[1])
        self._PhaseFeatures.append(shift[2])
        self._PhaseFeatures.append(shift[3])
        self._PhaseFeatures.append(shift[4])
        self._PhaseFeatures.append(shift[5])
        
        return np.asarray(self._PhaseFeatures)    

In [ ]:
#%%  Define class for frequency feature extraction ##
class Extract_Freq_Features:    
    def __init__(self, rawTimeData, rpm, Fs):
        self._FreqFeatures = []
        # Remove bias (subtract mean by each channel) #
        self._rawTimeData = rawTimeData - np.expand_dims(np.mean(rawTimeData,axis=1),axis=1) # Raw time data
        
        self._Fs = Fs                    # Sampling frequency [Hz]
        self._rpm = rpm/60               # RPM for every second [Hz]
       
    def FFT(self):
        # Perform FFT #
        _N = self._rawTimeData.shape[1]
        _dt = 1/self._Fs
        _yf_temp = np.fft.fft(self._rawTimeData)
        self._yf = np.abs(_yf_temp[:,:int(_N/2)]) / (_N/2)
        self._xf = np.fft.fftfreq(_N, d=_dt)[:int(_N/2)]
        
        return self._xf, self._yf
    
    def Freq_IDX(self):
        _xf,_yf = self.FFT()
        
        # Motor #
        # find frequency index of 1x #
        self._Freq_1x = self._rpm
        self._1x_idx_Temp = abs(_xf - self._Freq_1x).argmin()
        self._1x_idx_Temp = self._1x_idx_Temp - 2 + np.argmax(self._yf[0][np.arange(self._1x_idx_Temp-2, self._1x_idx_Temp+3)])
        self._1x_idx = np.arange(self._1x_idx_Temp-1, self._1x_idx_Temp+2)

        # find frequency index of 2x #
        self._Freq_2x = self._xf[self._1x_idx[1]] * 2
        self._2x_idx_Temp = abs(_xf - self._Freq_2x).argmin()
        self._2x_idx_Temp = self._2x_idx_Temp - 5 + np.argmax(self._yf[0][np.arange(self._2x_idx_Temp-5, self._2x_idx_Temp+6)])
        self._2x_idx = np.arange(self._2x_idx_Temp-1, self._2x_idx_Temp+2)
        
        # find frequency index of 3x #
        self._Freq_3x = self._xf[self._1x_idx[1]] * 3
        self._3x_idx_Temp = abs(_xf - self._Freq_3x).argmin()
        self._3x_idx_Temp = self._3x_idx_Temp - 5 + np.argmax(self._yf[0][np.arange(self._3x_idx_Temp-5, self._3x_idx_Temp+6)])
        self._3x_idx = np.arange(self._3x_idx_Temp-1, self._3x_idx_Temp+2)
        
        # find frequency index of 4x #
        self._Freq_4x = self._xf[self._1x_idx[1]]  * 4
        self._4x_idx_Temp = abs(_xf - self._Freq_4x).argmin()
        self._4x_idx_Temp = self._4x_idx_Temp - 5 + np.argmax(self._yf[0][np.arange(self._4x_idx_Temp-5, self._4x_idx_Temp+6)])
        self._4x_idx = np.arange(self._4x_idx_Temp-1, self._4x_idx_Temp+2)
        
        
        return (self._1x_idx, self._2x_idx, self._3x_idx, self._4x_idx)

    def Features(self):
        _xf, _yf = self.FFT()
        idx = self.Freq_IDX()
                
        # Freq-domain Features #
        self._1x_Feature = np.sum(_yf[:, idx[0]], axis=1) 
        self._2x_Feature = np.sum(_yf[:, idx[1]], axis=1) 
        self._3x_Feature = np.sum(_yf[:, idx[2]], axis=1) 
        self._4x_Feature = np.sum(_yf[:, idx[3]], axis=1)
                
        self._FreqFeatures.append(self._1x_Feature)
        self._FreqFeatures.append(self._2x_Feature)
        self._FreqFeatures.append(self._3x_Feature)
        self._FreqFeatures.append(self._4x_Feature)
        
        return np.asarray(self._FreqFeatures)

In [ ]:
def validation(filename):
        
    try:
        cur = pd.read_csv(filename, header=None, skiprows=9)
        cur = np.asarray(cur)[:,1:4].transpose()

        meta = pd.read_csv(filename, header=None, skiprows=4, nrows=1)
        rpm = int(meta[2])

        fs = pd.read_csv(filename, header=None, skiprows=6, nrows=1)
        Fs = int(fs[1])

        TimeFeatureExtractor = Extract_Time_Features(cur)
        features_time = TimeFeatureExtractor.Features().flatten()

        PhaseFeatureExtractor = Extract_Phase_Features(cur, Fs)
        features_phase = PhaseFeatureExtractor.Features().flatten()
        FreqFeatureExtractor = Extract_Freq_Features(cur, rpm, Fs)
        features_freq = FreqFeatureExtractor.Features().flatten()

        features = np.concatenate((features_time, features_phase, features_freq))
        data = pd.DataFrame(features).T

        data.columns = [
               'R_AbsMax', 'S_AbsMax', 'T_AbsMax', 'R_AbsMean', 'S_AbsMean','T_AbsMean',
               'R_P2P', 'S_P2P', 'T_P2P', 'R_RMS', 'S_RMS', 'T_RMS', 
               'R_Skewness', 'S_Skewness', 'T_Skewness', 'R_Kurtosis', 'S_Kurtosis', 'T_Kurtosis',
               'R_Crest', 'S_Crest', 'T_Crest', 'R_Shape', 'S_Shape', 'T_Shape',
               'R_Impulse', 'S_Impulse', 'T_Impulse',
               'RS_phase', 'ST_phase', 'TR_phase', 'RS_Level', 'ST_Level', 'TR_Level',
               'R_1x', 'S_1x', 'T_1x', 'R_2x', 'S_2x', 'T_2x',
               'R_3x', 'S_3x', 'T_3x', 'R_4x', 'S_4x', 'T_4x'
               ]

        data['WATT'] = meta[3]
        y = pd.read_csv(filename, header=None, skiprows=3, nrows=1)
        y = y[1]

    except (NameError, IndexError, TypeError, pd.errors.EmptyDataError):
        print('Source Data Error')
        
    else:
        df = data[['WATT', 'R_AbsMax', 'S_AbsMax', 'T_AbsMax', 'R_AbsMean', 'S_AbsMean','T_AbsMean',
                   'R_P2P', 'S_P2P', 'T_P2P', 'R_RMS', 'S_RMS', 'T_RMS', 
                   'R_Skewness', 'S_Skewness', 'T_Skewness', 'R_Kurtosis', 'S_Kurtosis', 'T_Kurtosis',
                   'R_Crest', 'S_Crest', 'T_Crest', 'R_Shape', 'S_Shape', 'T_Shape',
                   'R_Impulse', 'S_Impulse', 'T_Impulse',
                   'RS_phase', 'ST_phase', 'TR_phase', 'RS_Level', 'ST_Level', 'TR_Level',
                   'R_1x', 'S_1x', 'T_1x', 'R_2x', 'S_2x', 'T_2x',
                   'R_3x', 'S_3x', 'T_3x', 'R_4x', 'S_4x', 'T_4x']]

        #norm = np.load('current_norm.npy')#norm
        #X = (df - norm[0]) / norm[1]

        xgb_joblib = joblib.load('/content/drive/MyDrive/SFEP/dataset/current/current_xgb.pkl') 
        y_pred = xgb_joblib.predict(df)
        name = os.path.basename(filename)
        print("Filename:%s, Prediction:%s, Y_Target:%d" % (name, y_pred[0], y))
        
    return y_pred[0], y[0]

In [ ]:
def framework():
    print('########## 전류 유효 검증 시작 #############')
    print('시작시간(UCT+09:00)',datetime.datetime.now(pytz.timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M:%S"))
    print('_Python.version:', sys.version)
    print('_Scipy.version:',scipy.__version__)
    print('_XGBoost.version:',xgb.__version__)
    print('_Sklearn.version:',sklearn.__version__)

In [ ]:
framework()

########## 전류 유효 검증 시작 #############
시작시간(UCT+09:00) 2021-08-09 19:58:03
_Python.version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
_Scipy.version: 1.4.1
_XGBoost.version: 1.4.2
_Sklearn.version: 0.22.2.post1


In [ ]:
!pip install xgboost==1.4.2

     |████████████████████████████████| 166.7 MB 8.4 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import xgboost
print(xgboost.compat.__file__)
print(xgboost.compat.XGBoostLabelEncoder)

/usr/local/lib/python3.7/dist-packages/xgboost/compat.py
<class 'xgboost.compat.XGBoostLabelEncoder'>


In [17]:
datapath = glob.glob("/content/drive/MyDrive/SFEP/dataset/current" + '**/**/*.csv', recursive=True)

In [19]:
type(datapath)

list

In [ ]:
start = time.time()

#normal = 

# parser = argparse.ArgumentParser()
# parser.add_argument('--path', default='current', type=str)
# args = parser.parse_args()


#datapath = glob.glob("/content/drive/MyDrive/SFEP/dataset/current", recursive=True)

#datapath = glob.glob(args.path + '**/**/*.csv', recursive=True)



#random = random.sample(datapath, 100)

y_hat = []
y_target = []
for file in datapath:
    y_pred, y = validation(file)
    y_hat.append(y_pred)
    y_target.append(y)
    
target_names = ['0', '1', '2', '3', '4']
print('########## 전류데이터 유효검증 결과 ##########')
print(f"검증데이터수: {len(y_hat)} 개")

accuracy = accuracy_score(y_target, y_hat)
print('예측 정확도: {0:.4f}'.format(accuracy))
#print(classification_report(y_target, y_hat, target_names=target_names, digits=4)) 
print('종료시간(UCT+09:00)',datetime.datetime.now(pytz.timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M:%S"))

end = time.time()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Filename:STFCB-20201012-0105-0159_20201104_175008_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201104_174907_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201105_131805_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_122206_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_124905_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_122805_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_125105_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_122406_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_122205_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_124907_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_124906_002.csv, Prediction:0, Y_Target:0
Filename:STFCB-20201012-0105-0159_20201106_122807_002.csv, Prediction:0, Y_T

In [ ]:
elapsed_time = end-start
print(f"검증소요시간：{elapsed_time}초")

y_prediction = pd.DataFrame(y_hat)
y_prediction.to_csv('y_predict_current.csv', index=False)

검증소요시간：7.800654888153076초
